# Install, Paths and Parameters

In [1]:
import os
from pathlib import Path
import getpass
import numpy as np
import time
import torch
from torch import nn
from tqdm import tqdm
import random
import sys

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino, ViTWrapper
from src.model.data import create_loader
from src.model.eval import validate_network

# Custom imports
import torchattacks
from torchattacks import *
import torch.optim as optim
from torchvision import transforms as pth_transforms
from torchvision.utils import save_image

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')

ORI_PATH = Path(DATA_PATH, 'ori_data/validation')
ORI_LABEL_PATH = Path(ORI_PATH,'correct_labels.txt')
ORI_IMAGES_PATH = Path(ORI_PATH,'images')

In [2]:
INDEX_SUBSET = None

BATCH_SIZE = 40

DEVICE = 'cuda'

In [ ]:
#!python ../setup/collect_env.py

In [7]:
model, linear_classifier = get_dino(model_name='vit_small', patch_size=16, n_last_blocks=4, avgpool_patchtokens=False, device=DEVICE)

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights from dino_deitsmall16_pretrain/dino_deitsmall16_linearweights.pth.


# Load data

In [8]:
org_loader = create_loader(ORI_IMAGES_PATH, ORI_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, is_adv_training=True)
len(org_loader.dataset)

50000

# Wrap model

In [9]:
model_wrap = ViTWrapper(model, linear_classifier, DEVICE, n_last_blocks=4, avgpool_patchtokens=False)
model_wrap= model_wrap.to(DEVICE)

# Attack model and generate datasets

In [10]:
attacks = [
    #(PGD(model_wrap, eps=0.03, alpha=0.015, steps=20), 'pgd_03'),
    #(CW(model_wrap, c=50, lr=0.0031, steps=30), 'cw'),
    (PGDL2(model_wrap, eps=0.03, alpha=0.015, steps=20), 'pgd_l2_03')
]

In [ ]:
total = len(org_loader.dataset)

for atk, name in attacks:
    
    print("-"*70)
    print(atk)
    
    DN_PATH = Path(DATA_PATH, f'adversarial_data/{name}')
    DN_LABEL_PATH = Path(DN_PATH, 'validation/labels.txt')
    DN_IMAGES_PATH = Path(DN_PATH, 'validation/images/')
    
    if not os.path.exists(DN_IMAGES_PATH):
        os.makedirs(DN_IMAGES_PATH)

    correct = 0
    clean_correct = 0
    start = time.time()
    
    for images, labels, img_names in tqdm(org_loader):
        
        labels = labels.to(DEVICE)
        
        adv_images = atk(images, labels)
        
        with torch.no_grad():
            outputs = model_wrap(adv_images)

        _, pre = torch.max(outputs.data, 1)

        correct += (pre == labels).sum()
        
        for i in range(adv_images.shape[0]):
            save_image(adv_images[i,:,:,:], Path(DN_IMAGES_PATH, Path(img_names[i])))
            
        del images
        del adv_images
        del labels
        torch.cuda.empty_cache()

    print('Total elapsed time (sec): %.2f' % (time.time() - start))
    print('Accuracy against attack: %.2f %%' % (100 * float(correct) / total))

  0%|          | 0/556 [00:00<?, ?it/s]

----------------------------------------------------------------------
PGDL2(model_name=ViTWrapper, device=cuda:0, eps=0.03, alpha=0.015, steps=20, random_start=True, eps_for_division=1e-10, attack_mode=default, return_type=float)


100%|█████████▉| 554/556 [1:30:22<00:19,  9.78s/it]